In [1]:
import pandas as pd
import re
import json
import xml.etree.ElementTree as xet
import requests as req
import time
import glob
import csv
import regex
import tqdm
import pytz
import numpy as np
import io
import gzip
from bs4 import BeautifulSoup

In [2]:
def parse_sitemap(fn):

    xml = xet.parse(fn)
    xml_root = xml.getroot()
    news_list = []
    
    for news in xml_root:
        try:
            result = {}
            result['url'] = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}loc').text
            lastmod = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}lastmod')
            if lastmod is not None:
                result['lastmod'] = lastmod.text

            #title = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}news:title').text
            #keywords = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}news:keywords').text
            #news_list.append(dict(zip(['url', 'lastmod'], [url, lastmod])))
            news_list.append(result)
        except:
            continue

    return news_list

In [3]:
sitemap_urls = parse_sitemap(io.StringIO(req.get('https://www.currenttime.tv/sitemap.xml').content.decode()))
sitemap_urls

[{'url': 'https://www.currenttime.tv/sitemap_54_1.xml.gz',
  'lastmod': '2020-12-13T02:14:17.8897107Z'},
 {'url': 'https://www.currenttime.tv/sitemap_54_2.xml.gz',
  'lastmod': '2020-12-13T02:14:20.7274475Z'},
 {'url': 'https://www.currenttime.tv/sitemap_54_latest.xml.gz'},
 {'url': 'https://www.currenttime.tv/sitemap_54_news.xml.gz'},
 {'url': 'https://www.currenttime.tv/sitemap_54_sections.xml.gz'},
 {'url': 'https://www.currenttime.tv/sitemap_54_videos.xml.gz'}]

In [4]:
news_db = []

for fn in tqdm.tqdm(sitemap_urls):
    try:
        with gzip.open(io.BytesIO(req.get(fn['url']).content)) as f:
            sitemap = io.StringIO(f.read().decode())
        news = parse_sitemap(sitemap)
    except Exception as e:
        print(e)
        continue
    news_db = news_db + news

100%|██████████| 6/6 [00:05<00:00,  1.18it/s]


In [5]:
df = pd.DataFrame(news_db)
df['lastmod'] = pd.to_datetime(df['lastmod'], format='%Y-%m-%dT%H:%M:%S', utc=True).dt.round('1s')
df.head()

,url,lastmod
0,https://www.currenttime.tv/a/glavnoe-chto-ih-n...,2020-12-12 16:10:08+00:00
1,https://www.currenttime.tv/a/subbotnie-gulyani...,2020-12-12 13:08:30+00:00
2,https://www.currenttime.tv/a/30997271.html,2020-12-12 11:19:01+00:00
3,https://www.currenttime.tv/a/verkhovny-sud-ssh...,2020-12-12 10:28:25+00:00
4,https://www.currenttime.tv/a/umer-valentin-gaf...,2020-12-12 09:25:34+00:00


In [7]:
df[(df.lastmod >= '2019-11-01')].to_json('news_currenttime.json', date_format='iso', date_unit='s')
df[(df.lastmod >= '2019-11-01')].shape

(8852, 2)

# Parsing

In [17]:
def parse_news(text):
    soup = BeautifulSoup(text, features='lxml')
    news_page = soup.find(name="div", attrs={'id': "article-content"})
    news_text = ''
    
    if news_page:
        bad = news_page.find_all(name=['script', 'iframe', 'svg', 'figure'])
        if bad:
            [tag.decompose() for tag in bad]
            
        news_elems = news_page.find_all(name=['p', 'li', 'dl'])
        for news_elem in news_elems:
            news_text += " " + news_elem.get_text(" ", strip=True)
        news_text = re.sub(r'[\n\r]+', ' ', news_text).replace(u'\xa0', u' ').replace(u'\u200b', u' ')

    description_elem = soup.find(name="meta", attrs={'name': "description"})
    description = re.sub(r'[\n\r]+', ' ', description_elem.get('content')).replace(u'\xa0', u' ') if description_elem else ''
    
    tags_elem = soup.find(name="meta", attrs={'name': "keywords"})
    tags = re.sub(r'[\n\r]+', ' ', tags_elem.get('content')).replace(u'\xa0', u' ') if tags_elem else ''
    
    return {'news_text': news_text, 'tags': tags, 'description': description}

In [18]:
parse_news(req.get(df.iloc[1000]['url']).text)

{'news_text': ' Жители Чукотки традиционно добывают для своего пропитания китов, моржей, нерпу и рыбу. Рыболовство и китобойный промысел на крайнем севере –  занятие рискованное и опасное. Почти каждый год кто-нибудь из рыбаков не возвращается домой с промысла. А с тех пор как российское законодательство начало регулировать добычу водных биоресурсов, выживать коренным народам Севера стало еще труднее. Сибирь.Реалии рассказывают историю села, где квоты на ловлю рыбы стал вопросом выживания. До 2015 года жители Чукотки занимались добычей рыбы и морских млекопитающих без ограничений. Летом добывали китовое и моржовое мясо, а также ловили впрок рыбу, чтобы запасов хватало на всю долгую полярную зиму. Рыбу местные жители замораживают, солят и коптят для себя, икру засаливают и продают, чтобы покупать в магазинах продукты, о стоимости которых говорят с возмущением. Булка хлеба здесь стоит 75 рублей, яйца – 365 рублей за десяток, а в некоторых селах – вообще по 700 рублей за десяток. Говорят,

In [28]:
import pandas as pd
pd.to_datetime(pd.Series(1560000000), unit='s').dt.strftime("%Y-%m-%d %H:%M:%S").item()

'2019-06-08 13:20:00'